## AI Inference on a local camera
This notebook is a simple example how to use DeGirum PySDK to do AI inference of a video stream from local camera.
OpenCV is required to run this sample.

This script works with

0. DeGirum Cloud Server,
1. AI server equipped with DeGirum ORCA accelerator shared via Peer-to-Peer VPN,
2. AI server equipped with DeGirum ORCA accelerator running in local network and
3. AI server equipped with DeGirum ORCA accelerator running on the same machine as this code.

To try different options, you just need to change the __model_zoo_option_index__ in the code below.

The script needs a web camera connected to the machine running this code. The __camera_index__ also needs to be specified in the code below.

### Specify your model zoo option and camera index here

In [57]:
model_zoo_option_list = ['DEGIRUM_CLOUD_SERVER', 'P2P_VPN_SERVER', 'LOCAL_NETWORK_SERVER', 'LOCALHOST']
model_zoo_option_index=1
camera_index=1

### The rest of the cells below should run without any modifications

In [58]:
import degirum as dg # import DeGirum PySDK
import cv2 # OpenCV
import os
from dotenv import load_dotenv

envs_loaded=load_dotenv()
model_zoo_option_list = ['DEGIRUM_CLOUD_SERVER', 'P2P_VPN_SERVER', 'LOCAL_NETWORK_SERVER', 'LOCALHOST']

In [59]:
# Some helper functions

def show(img, capt = "<image>"):
    # show opencv image
    cv2.imshow(capt, img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('x') or key == ord('q'):
        raise KeyboardInterrupt

In [60]:
model_zoo_option=model_zoo_option_list[model_zoo_option_index]
if model_zoo_option=='DEGIRUM_CLOUD_SERVER':
    zoo = dg.connect_model_zoo(os.getenv('DEGIRUM_CLOUD_SERVER_ADDRESS'), token=os.getenv('DEGIRUM_CLOUD_TOKEN'))
elif model_zoo_option=='P2P_VPN_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('P2P_VPN_SERVER_ADDRESS'))
elif model_zoo_option=='LOCAL_NETWORK_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_NETWORK_SERVER_ADDRESS'))
elif model_zoo_option=='LOCALHOST':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_HOST_ADDRESS'))
else:
    print('Model zoo option not supported \n')               

In [61]:
# load AI model 'ssd_mobilenet_v2' for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1")

# set model parameters
model.image_backend = 'opencv' # select OpenCV backend: needed to have overlay image in OpenCV format
model.input_numpy_colorspace='BGR'
model.overlay_show_probabilities = True # show probabilities on overlay image
model._model_parameters.InputImgFmt=['JPEG']

In [62]:
# open video stream from local camera 
stream = cv2.VideoCapture(camera_index)
if (stream.isOpened()== False):
    print("Error opening video stream")
else:
    print("Succesful in opening video stream")

Succesful in opening video stream


In [63]:
# define iterator function, which returns frames from camera 
def source():
    while True:
        ret, frame = stream.read()
        yield frame

In [64]:
# AI prediction loop
# Press 'x' or 'q' to stop
try:
    for res in model.predict_batch(source()):
        show(res.image_overlay,"AI Camera")    
except KeyboardInterrupt:
    pass # ignore KeyboardInterrupt errors
finally:
    cv2.destroyAllWindows() # close OpenCV windows

stream.release() # release camera stream